In [2]:
import paramiko
import io
import os

from dotenv import load_dotenv

load_dotenv()

hostname = os.getenv("SFTP_HOST")
port = int(os.getenv("SFTP_PORT"))
username = os.getenv("SFTP_USER")
password = os.getenv("SFTP_PASSWORD")

if not password:
    raise ValueError("❌ SFTP_PASSWORD nicht in .env definiert!")

print(f"🔄 Verbinde zu {hostname}:{port} als {username}...")

try:
    transport = paramiko.Transport((hostname, port))
    transport.start_client()
    transport.auth_password(username=username, password=password)

    if not transport.is_authenticated():
        raise paramiko.AuthenticationException("Auth failed")

    print("✅ SFTP-Verbindung erfolgreich!")

    sftp = paramiko.SFTPClient.from_transport(transport)
    sftp.chdir("out")

    # === LV.ZIP HERUNTERLADEN ===
    print("📥 Lade lv.zip...")
    lv_zip_data = io.BytesIO()
    sftp.getfo("lv.zip", lv_zip_data)
    lv_zip_data.seek(0)
    print(f"  ✓ {len(lv_zip_data.getvalue()) / (1024**2):.1f} MB")

    # === AKI.ZIP HERUNTERLADEN ===
    print("📥 Lade aki.zip...")
    aki_zip_data = io.BytesIO()
    sftp.getfo("aki.zip", aki_zip_data)
    aki_zip_data.seek(0)
    print(f"  ✓ {len(aki_zip_data.getvalue()) / (1024**2):.1f} MB")

    sftp.close()
    transport.close()
    print("✅ Download abgeschlossen!")

except paramiko.AuthenticationException as e:
    raise ValueError(f"❌ Authentifizierung fehlgeschlagen: {e}")
except Exception as e:
    raise ConnectionError(f"❌ SFTP-Fehler: {e}")

🔄 Verbinde zu ftp.bruder-gruppe.de:22 als hochschuleog...
✅ SFTP-Verbindung erfolgreich!
📥 Lade lv.zip...
  ✓ 440.9 MB
📥 Lade aki.zip...
  ✓ 692.1 MB
✅ Download abgeschlossen!


In [3]:
import zipfile
import subprocess

os.makedirs("./data", exist_ok=True)

# === LV.ZIP ENTPACKEN ===
print("📦 Entpacke lv.zip...")
try:
    with zipfile.ZipFile(lv_zip_data, 'r', allowZip64=True) as z:
        z.extractall("./data")
    print("  ✓ Erfolgreich entpackt")
except NotImplementedError:
    print("  ⚠️ ZIP-Kompression nicht unterstützt, nutze Kommandozeile...")
    lv_zip_data.seek(0)
    with open("/tmp/lv.zip", "wb") as f:
        f.write(lv_zip_data.read())
    subprocess.run(["unzip", "-o", "/tmp/lv.zip", "-d", "./data"], check=True)
    print("  ✓ Mit unzip entpackt")

# === AKI.ZIP ENTPACKEN ===
print("📦 Entpacke aki.zip...")
try:
    with zipfile.ZipFile(aki_zip_data, 'r', allowZip64=True) as z:
        z.extractall("./data")
    print("  ✓ Erfolgreich entpackt")
except NotImplementedError:
    print("  ⚠️ ZIP-Kompression nicht unterstützt, nutze Kommandozeile...")
    aki_zip_data.seek(0)
    with open("/tmp/aki.zip", "wb") as f:
        f.write(aki_zip_data.read())
    subprocess.run(["unzip", "-o", "/tmp/aki.zip", "-d", "./data"], check=True)
    print("  ✓ Mit unzip entpackt")

print("✅ Alle Dateien entpackt!")

📦 Entpacke lv.zip...
  ✓ Erfolgreich entpackt
📦 Entpacke aki.zip...
  ⚠️ ZIP-Kompression nicht unterstützt, nutze Kommandozeile...
Archive:  /tmp/aki.zip
  inflating: ./data/aki/art.csv      
  inflating: ./data/aki/lv.csv       
  ✓ Mit unzip entpackt
✅ Alle Dateien entpackt!


In [6]:
import pandas as pd

# === DATAFRAMES LADEN ===
dataframes = {}

print("\n📊 Lade DataFrames...")

# LV DataFrame
lv_path = "data/lv/lv.csv"
if os.path.exists(lv_path):
    dataframes['lv'] = pd.read_csv(lv_path, low_memory=False)
    print(f"  ✓ lv.csv: {len(dataframes['lv'])} Zeilen")
else:
    print(f"  ❌ {lv_path} nicht gefunden!")

# ART DataFrame
art_path = "data/aki/art.csv"
if os.path.exists(art_path):
    dataframes['art'] = pd.read_csv(art_path, low_memory=False)
    print(f"  ✓ art.csv: {len(dataframes['art'])} Zeilen")
else:
    print(f"  ❌ {art_path} nicht gefunden!")

print(f"\n🎉 Fertig! {len(dataframes)} Datensätze geladen.")
print(f"   Verfügbare DataFrames: {list(dataframes.keys())}")


📊 Lade DataFrames...
  ✓ lv.csv: 5940224 Zeilen
  ✓ art.csv: 42915052 Zeilen

🎉 Fertig! 2 Datensätze geladen.
   Verfügbare DataFrames: ['lv', 'art']


In [ ]:
# Setup & Normalisierung
import os
import re
from dotenv import load_dotenv

load_dotenv()

LV_OUT_PATH = os.getenv("LV_OUT_PATH", "data/lv_clean.csv")
NROWS = int(os.getenv("LV_NROWS", "0")) or None

if 'lv' in dataframes:
    df = dataframes['lv'].copy()
else:
    raise ValueError("'lv' nicht in geladenen Daten gefunden!")

required_cols = {"id", "oz_bez"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Fehlende Spalten: {missing}")

TRIGGER_PATTERNS = [
    r"\bwie\s+(?:oben|vor|zuvor|zuletzt)\b",
    r"\bw(?:ie)?\s*o\b",
    r"\bdito\b",
    r"\bentspr\.\s*oben\b",
    r"\bgleich\s+wie\b",
    r"\bwie\s+beschr(?:ieben|.)\b",
    r"\bwie\s+pos(?:ition)?\s*\d+\b",
    r"\bwie\s+z(?:ur|u)\s+pos\b",
    r"\bdesgleichen\b",
]
trigger_re = re.compile("|".join(f"(?:{p})" for p in TRIGGER_PATTERNS), flags=re.IGNORECASE)

def has_trigger(text: str) -> bool:
    return bool(trigger_re.search(str(text)))

# Neue Spalten zum Inspizieren
df["Referenz"] = ""
df["Original"] = ""

print("✓ Normalisierung abgeschlossen")
print(f"  Arbeite mit {len(df)} Zeilen aus 'lv'")

In [ ]:
# Referenzen auflösen & Speichern

first_by_id = df.sort_values("id").groupby("id", as_index=True)["oz_bez"].first()

def resolve_reference(current_id: int):
    search_id = current_id - 1
    while search_id in first_by_id.index:
        t = first_by_id.loc[search_id]
        if not has_trigger(t):
            return t
        search_id -= 1
    return None

def resolve_row(row):
    text = row["oz_bez"]
    row["Original"] = text

    if not has_trigger(text):
        row["Referenz"] = ""
        return row

    try:
        cid = int(row["id"])
    except Exception:
        row["Referenz"] = ""
        return row

    base = resolve_reference(cid)
    if base:
        row["Referenz"] = base
        text = f"Base: {base}\n\nText: {text}"


    text = re.sub(r"\s{2,}", " ", text)
    row["oz_bez"] = text
    return row

df = df.apply(resolve_row, axis=1)

if "arnr" in df.columns:
    df["arnr"] = df["arnr"].astype(str).str.strip()

os.makedirs(os.path.dirname(LV_OUT_PATH), exist_ok=True)
df.to_csv(LV_OUT_PATH, index=False, encoding="utf-8")

print(f"✓ Fertig: {LV_OUT_PATH}")
print(f"  Spalten: Referenz | Original | oz_bez (kombiniert)")

In [ ]:
df.info()
